# Converting Slippi to a DataFrame
## Table of Contents
1. [Problem Statement](#problem)
2. [Context](#context)<br>
    a. [Terminology](#terms)<br>
    b. [Why Super Smash Bros.?](#why)<br>
    c. [How does Super Smash Bros. Work?](#how)<br>
3. [Executive Summary](#executive)<br>
    a. [Data Gathering](#gather)<br>
    b. [Parsing Data](#parse)<br>
    c. [Modeling](#modeling)<br>
    d. [Limitations](#limitdoesnotexist)<br>
4. [Importing Libraries](#imports)
5. [Retrieving Filepaths of Each Slippi Game](#filepaths)

<a id='problem'></a>
## Problem Statement

Note: For those unfamiliar with Super Smash Bros., I encourage you to read the [context](#context) section.

Fox versus Falco is a frequent match in competitive Melee. Discussions about whether Fox or Falco have the upperhand in the matchup is a popular debate among players. I believe that among today's top ranked players in the world, Fox will most likely win because of his faster speed over Falco. However, Falco is more likely to win against newer players because he is able to utilize different tools to perform combos that lead to Fox losing a stock.

Since Falco is often one of the first major hurdles for new players to defeat, I hope to construct a coaching tool that will help them overcome this. The final product will use a collection of Slippi games as input and provide summary statistics on the games. For coaching tools of the product, a player can feed in a collection of games to feed as input to learn how a specified player and character plays. Then this will create an AI that someone can battle. This is useful for those who wish to practice against a live player, but does not have immediate access to others. A similar idea to how Super Smash Bros. Wii U utilized [Amiibos](https://www.youtube.com/watch?v=uOnLcVOvrEE). As a quick check to see how feasible this project is, I will train a recurrent neural network on a series of games to see how well the network can learn Fox's wake-up behavior.
<img src="../images/melee-wallpaper.jpg" alt="Drawing" style="width: 600px;"/>

<a id='context'></a>
## Context
<a id='why'></a>
### Why Super Smash Bros.?

Super Smash Bros. is a video game series published by Nintendo where video game characters of different franchises battle it out. The second game of the series, Super Smash Bros. Melee, was released in December 2001. This sparked fun parties, happy players, and heated rivalries. These rivalries grew from friend groups, to local neighborhood challenges, to large-scale tournaments by 2002. If you are interested in learning the history of the Super Smash Bros. Melee competitive scene, I encourage you to watch a docuseries called [The Smash Brothers](https://www.youtube.com/watch?v=NSf2mgkRm7Q&list=PLoUHkRwnRH-IXbZfwlgiEN8eXmoj6DtKM) produced by East Point Pictures.

Many players within this community are dedicated to becoming the best player that they can be. There are many fan-made tools and mods to the game for the purpose of either improving the player experience in training or improving the production of content. Some tools allow players to create save states so that they can easily practice a scenario quickly. Others allow content creators to create replays for the audience to enjoy. Today, I would like to highlight one of these tools, Project Slippi.
- [Website](https://slippi.gg/)
- [Github](https://github.com/project-slippi/project-slippi)
- [Medium](https://medium.com/project-slippi)

<a id='how'></a>
### How Does Super Smash Bros. Work?

In this project, I will be only looking at 1v1 tournament legal matches. The criteria for that are:
- The match must be 1v1 with no other players or CPU's in the match.
- The match must be on a tournament legal stage.
- The match must be 8 minutes or less.

In a 1v1 match, players are limited to defeat their opponent with nothing more than the abilities of themselves as a player and the abilities of their selected character. Characters have various moves such as jabs, smash attacks, tilt attacks, special attacks, grabs, and aerial attacks. Most of these attacks have slight variations of the same attack, but at different directions. When a player strikes their opponent with an attack, then the opponent's damage percentage goes up.
<img src="../images/damage-example.gif"/> <center>Fox Damaging Falco</center>

As a character's damage increases, then the distance at which they are launched after a hit then increases. This is beneficial as an opponent because the further they travel for each hit, then it should be easier to push them through a blast zone and have them lose a stock. Once a character loses all their stocks, then the other player is determined the winner. If a timeout occurs, then the player with the most stocks and least damage wins. In the case that those are tied as well, then a rematch is played with each charcter getting one stock.

<img src="../images/GAME.gif"/> <center>Jigglypuff Defeating Fox's Last Stock; Winning the Game</center>

### A Primer on Fighting Games

Fighting games is one of the many genres of video games. Some fighting game titles that may sound familiar are Street Fighter and Tekken. These games are unique in that Street Fighter is a 2-dimensional fighter where characters can only move left or right and Tekken characters can move forward, backwards, left or right. While all of these games are systematically different, they all share similar strategies that a player can utilize. For example, players can play as a heavy brute that can pack a punch, but are slow. Or they can play a glass cannon that can attack with lightning fast speed, but they are easy to kill once their momentum is broken.

The idea of mixups is another common thread among fighting games. One of the most frequent mixup situations that occur in fighting games are when a character is laying on the ground. Usually, the character has one of four options known as get-up options or wake-up options. They can stay on the ground, get up, roll forward, or roll back. The opponent must be able to either predict or quickly react to whichever option the player takes and act accordingly. If the opponent does not capitalize on the player's vulnerability, then the player has a chance to fight back and win.

In Melee, when a character is launched towards the floor, wall, or ceiling, then they have the option to tech. When a player is hit and placed in a situation where they must decide whether to tech in-place, tech-roll left, or tech-roll right, they may miss the 20 frame window and not tech. This will cause them to stay vulnerable on the ground with few options to retaliate. However, like stated before, this is a mixup. If a player frequently techs to the right, then their opponent will be accustomed to that behavior. So whenever the the opponent sees that the player has the option to tech, then they will assume the player will tech right and adjust the combo. If the player suddenly techs in-place and the opponent did not expect that, then the player has recovered safely and is free to move. This process is known as tech chasing - the opponent is tech-chasing the player. 
![tech-chasing](../images/tech-chase.gif) <center>Captain Falcon tech-chasing Fox McCloud</center>



<a id='executive'></a>
## Executive Summary
<a id = 'gather'></a>
### Data Gathering
To begin, I considered constructing a script that would scrape [Slippi's site](https://slippi.gg) for games that occured during a tournament using the library `selenium`. I opted not to do this because the official Slippi Discord channel has the `!replaydumps` chat command that provides a download link to Slippi files from [Fight Pitt 9](https://smash.gg/tournament/fight-pitt-9-1/details), [Full Bloom 5](https://smash.gg/tournament/full-bloom-5/details), [The Gang Steals the Script](https://smash.gg/tournament/the-gang-steals-the-script/details), and [Pound 2019](https://smash.gg/tournament/pound-2019/details). The source of the data is on a different platform, but each are controlled by the creators and major contributors to the project such as [Fizzi](https://twitter.com/Fizzi36).

<a id='parse'></a>
### Parsing Data
With the data in hand, I used the `slippi` library to read in each file as a Slippi's own Game object. Game objects have attributes whose values are sometimes other objects. Since each Slippi file has the same structure, I created a function `metadata_to_df` to parse the metadata of each game. The objective here is to filter the games as needed. For example, I want 1v1 games, so I can filter for games where the team battle option is set to off.

<a id = 'modeling'></a>
### Modeling
Once I have filtered the games I will use as input, I will then create another function to parse the Frame objects of each game. Each Frame object contains information of each frame within the game such as character position and controller inputs. These values will be fed to a recurrent neural network (RNN). The RNN will have a simple topology of a single hidden layer. This is because I am interested to see how well the RNN can learn the players behavior the least amount of complexity possible. If the results were not much better than the baseline accuracy score, then I would increase the complexity of the topology, but cannot due to the below limitations.

<a id = 'limitatdoesnotexist'></a>
### Limitations
tl;dr Power and money.

When parsing the metadata and frame data from each game, it took a considerable amount of time to execute for all Fight Pitt 9 games. This was a concern because Fight Pitt 9 contained the least amount of games compared to the other tournaments. This lack of power encouraged me only utilize Fight Pitt 9 games.

Since every frame of a game contains multiple features that can be used as inputs to the RNN, there is a lot of data to deal with. If a single game lasts for a minute, then that is 3600 frames (60 seconds * 60 frames per second). Each frame contains 333 features after cleaning and dummying. In short, a lot of power is needed to be able to fit the neural network quickly.

I could use AWS cloud computing to perform the task, but the machine's that were noticeably stronger than my machine cost too much for me at the moment.

<a id = 'imports'></a>
## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import slippi as slp
import os

<a id = 'filepaths'></a>
## Retrieving Filepaths of Each Slippi Game
Create a list that contains each file path to all games within the provided directory<sub>[1](https://kite.com/python/examples/4286/os-get-the-path-of-all-files-in-a-directory)</sub>.

In [2]:
# Directory to take filepaths from
dir_fp9 = '../../data/Fight-Pitt-9/'

# lists of file paths to Slippi files
fight_pitt_9, full_bloom_5, gang = [], [], []

# adding each file to their respective list
# Fight Pitt 9
for path in os.listdir(dir_fp9):
    full_path = os.path.join(dir_fp9, path)
    if os.path.isfile(full_path):
        fight_pitt_9.append(full_path)

According to Finder, there are 1,151 items in the Fight Pitt 9 directory.

In [3]:
print(f'# of Fight Pitt 9 filepaths: {len(fight_pitt_9)}')
print('Expecting 1151')
fight_pitt_9[:5]

# of Fight Pitt 9 filepaths: 1151
Expecting 1151


['../../data/Fight-Pitt-9/Game_20190406T182021.slp',
 '../../data/Fight-Pitt-9/Game_20190406T054329.slp',
 '../../data/Fight-Pitt-9/Game_20190406T113710.slp',
 '../../data/Fight-Pitt-9/Game_20190406T060932.slp',
 '../../data/Fight-Pitt-9/Game_20190406T063208.slp']

### Data Normalization: 1st Normal Form<sub>[2](https://www.guru99.com/database-normalization.html#2)</sub>
You may have noticed that some ` copy` before the file extension if you looked at more game files within the list. This occurred because the main directory that contained all Slippi files for a particular tournament grouped all files according to which station (i.e. console) at which the game was played.

In other words, the main directory contained sub-directories that held the games. In order to get all games in one folder, regardless of station, I had moved all Slippi files into the main directory and deleted the subdirectories once emtpy. Since it is possible for two stations to have started a game at the same time within a second, then it is possible for a `game_id` to be shared between multiple games. Currently, the filename of each Slippi file will be used as the index in the metadata  dataframe. This creates an anomoly in which the index that represents a single game may be used to represent two games.

In future versions, I will adjust [cell 2](#cell2) to be able to retrieve the filepaths of Slippi files within all sub-directories and then make the index of the metadata dataframe the filename of the Slippi file as well as the station ID.

These anomalies are not addressed at the moment because I will only be using games that are Fox vs. Falco on Final Destination. Of all games that fit that criteria, none of them share a filename.

## Reading in a Slippi File

In [4]:
# A single game from each tournament:
game = slp.Game(fight_pitt_9[0])

## 2. Extracting Metadata From Games

### Date
When reading in a list of Slippi files, a list comprehension will be used to iterate through each game.

In [5]:
game.metadata

Metadata(date=2019-04-06 18:20:21+00:00, duration=11653, platform=Platform.NINTENDONT, players=(None, None, None, None))

In [6]:
# A Game object has a metadata attribute whose value is a Metadata object.
# This Metadata object has attributes shown below.
date = game.metadata.date
print(date)

2019-04-06 18:20:21+00:00


### Duration
This details the length of the match in _n_ frames where a single frame is 1/60 seconds.

In [7]:
duration = game.metadata.duration
duration

11653

### Platform
The platform on which the game was played. Either on a Dolphin emulator or console.

In [8]:
platform = game.metadata.platform
platform

Platform.NINTENDONT

### Characters
We will need to determine which controller ports are being used to determine where to read data from.

In [9]:
game.metadata.players

(None, None, None, None)

It appears this data is not stored in the metadata attribute. Since these files are able to reconstruct a replay of the game using the controller inputs, then this information must be somewhere in the file. That "somewhere" is the `start` attribute.

In [10]:
game.start

Start(is_frozen_ps=None, is_pal=False, is_teams=False, players=(Player(character=CSSCharacter.ICE_CLIMBERS, costume=0, stocks=4, tag=, team=None, type=Type.HUMAN, ucf=UCF(dash_back=DashBack.UCF, shield_drop=ShieldDrop.UCF)), None, None, Player(character=CSSCharacter.MARTH, costume=1, stocks=4, tag=, team=None, type=Type.HUMAN, ucf=UCF(dash_back=DashBack.UCF, shield_drop=ShieldDrop.UCF))), random_seed=3456179710, slippi=Slippi(version=1.7.1), stage=Stage.FINAL_DESTINATION)

In [11]:
game.start.players

(Player(character=CSSCharacter.ICE_CLIMBERS, costume=0, stocks=4, tag=, team=None, type=Type.HUMAN, ucf=UCF(dash_back=DashBack.UCF, shield_drop=ShieldDrop.UCF)),
 None,
 None,
 Player(character=CSSCharacter.MARTH, costume=1, stocks=4, tag=, team=None, type=Type.HUMAN, ucf=UCF(dash_back=DashBack.UCF, shield_drop=ShieldDrop.UCF)))

In [17]:
[game.start.players.index(port) if port != None else np.nan for port in game.start.players]

[0, nan, nan, 3]

In [16]:
# Similar to the Metadata, each Game object has a
# Start object stored in each game's start attribute.
ports = [game.start.players.index(port) for port in game.start.players if port != None]
ports

[0, 3]

In [13]:
game.start.players[0].character

CSSCharacter.ICE_CLIMBERS

In [15]:
characters = [game.start.players[port].character for port in ports]
characters

[CSSCharacter.ICE_CLIMBERS, CSSCharacter.MARTH]

### Stage

In [ ]:
game.start.stage

#### For Older Versions of Slippi Such as Fight Pitt 9

### Metadata Series
When reading in multiple games a DataFrame will be made where each row will represent a single game.

In [ ]:
pd.Series({'date': date,
           'duration': duration,
           'platform': platform,
           'p1_port': ports[0],
           'p1_character': characters[0],
           'p2_port': ports[1],
           'p2_character': characters[1],
          'stage': game.start.stage})

### Getting all Metadata

In [ ]:
test_fp9 = fight_pitt_9[:10]

In [ ]:
game.start.players[0]

In [ ]:
fight_pitt_9[0].split('/')[-1].strip('Game_').strip('.slp')

In [ ]:
### THIS IS ADJUSTED TO TAKE ANY NUMBER OF FILE PATHS
### AND PARSE TO A DATAFRAME
### INCLUDING PORT AND CHARACTER INFO
def metadata_to_df(slp_paths):
    '''
    Of a collection of games, store the metadata as a dataframe.
    
    slp_paths (list): each value is the file path to games
    returns a dataframe
    '''
    length = len(slp_paths)
    count = 0
    dates, game_id, durations, plats, p1_ports, p1_chars, p2_ports, p2_chars, stages, is_teams, is_pal = list(), \
    list(), list(), list(), list(), list(), list(), list(), list(), list(), list()
    for path in slp_paths:
        count += 1
        print(f'Parsing metadata from file {count} of {length}: {round(count / length * 100, 2)}%', end = '\r')
        # try to create Game object, else skip it and try the next one
        try:
            game = slp.Game(path)
        except:
            print(f'Skip game {count} of {length}')
            continue
            
        # set game ID
        # to get file path using game_id:
        # ../folder_directory/Game_[game_id].slp
        game_id.append(slp_paths[count - 1].split('/')[-1].strip('Game_').strip('.slp'))
        
        # take the date, duration, and platform data
        dates.append(game.metadata.date)
        durations.append(game.metadata.duration)
        plats.append(game.metadata.platform)

        # get active ports
        ports = [game.start.players.index(port) for port in game.start.players if port != None]
        p1_ports.append(ports[0])
        p2_ports.append(ports[1])

        # get characters
        characters = [game.start.players[port].character for port in ports]
        p1_chars.append(characters[0])
        p2_chars.append(characters[1])
        
        # get stages played on
        stages.append(game.start.stage)
        
        # is the game not a 1v1
        is_teams.append(game.start.is_teams)
        
        # is this not a v1.02 match
        is_pal.append(game.start.is_pal)
        
    return pd.DataFrame(data = {
            'game_id': game_id,
            'date': dates,
            'duration': durations,
            'platform': plats,
            'p1_port': p1_ports,
            'p1_char': p1_chars,
            'p2_port': p2_ports,
            'p2_char': p2_chars,
            'stage': stages,
            'is_teams': is_teams,
            'is_pal': is_pal
        })

Even though the character information is missing in the metadata, the character information is stored in other locations of each game file such as the start attribute of the Game object.

In [ ]:
game.start.players[0].ucf

In [ ]:
df_fp9 = metadata_to_df(fight_pitt_9)
df_fp9.head()

In [ ]:
df_fp9.to_csv('../data/fp9.csv')

# END OF NOTEBOOK 1

In [ ]:
sum(df_fp9['duration'])

In [ ]:
df_fp9.shape == df_fp9.drop_duplicates(subset = ['game_id']).shape

In [ ]:
df_fp9.shape

In [ ]:
df_fp9.set_index('game_id', inplace = True)

For an easier time determining which character each player used in the game, I will be using the [documentation](https://py-slippi.readthedocs.io/en/latest/source/slippi.html) to make sure they are appropriately mapped. Upon looking into the docs, I noticed that there are two enumeration objects regarding characters, `CSSCharacter` and `InGameCharacter`. These objects label all tournament legal chracters, but in different orders. For example, Mario has a value of 0 in the `InGameCharacter` object, but has a value of 8 in `CSSCharacter`. Since I know which characters are more frequently played in tournaments, I'll take the value counts of one character column and determine if the `CSSCharacter` was interpretted or `InGameCharacter`.

In [ ]:
df_fp9['p1_char'].value_counts()

# MAKE THIS A TABLE
Val. `Object`: Character

    `Other Object`: Character
    
9. `CSSCharacter`: Marth

    `InGameCharacter`: Peach
    
Since both characters are popular relative to the rest of the characters, I am not completely certain that a value of 9 represents Marth or Peach.

20. `CSSCharacter`: Fox

    `InGameCharacter`: Young Link

If someone were to tell me the second most frequent character in ports 0, 1, or 2 are either Fox or Young Link, then I would am confident that the it was Fox. I am now inclined to say that the values represent the `CSSCharacter` object rather than the `InGameCharacter` object.

2. `CSSCharacter`: Fox

    `InGameCharacter`: Captain Falcon

Both of these characters are popular, so I am again uncertain.

0. `CSSCharacter`: Captain Falcon

    `InGameCharacter`: Mario
    
This is a similar comparison to Fox and Young Link. Of Captain Falcon and Mario, Captain Falcon is the more popular character. Additionally, comparing Fox and Captain Falcon, it is expected that there are more Fox players than Captain Falcon players.

With the above comparisons, I am very confident that the values are the `CSSCharacter` object.

In [ ]:
csscharacter = {
    0: 'Captain Falcon',
    1: 'Donkey Kong',
    2: 'Fox',
    3: 'Game and Watch',
    4: 'Kirby',
    5: 'Bowser',
    6: 'Link',
    7: 'Luigi',
    8: 'Mario',
    9: 'Marth',
    10: 'Mewtwo',
    11: 'Ness',
    12: 'Peach',
    13: 'Pikachu',
    14: 'Ice Climbers',
    15: 'Jigglypuff',
    16: 'Samus',
    17: 'Yoshi',
    18: 'Zelda',
    19: 'Sheik',
    20: 'Falco',
    21: 'Young Link',
    22: 'Dr. Mario',
    23: 'Roy',
    24: 'Pichu',
    25: 'Ganondorf'
}

In [ ]:
df_fp9['p1_char_name'] = df_fp9['p1_char'].map(csscharacter)
df_fp9['p2_char_name'] = df_fp9['p2_char'].map(csscharacter)

In [ ]:
df_fp9.head()

In [ ]:
df_fp9.shape

Get only the filepaths of games that are not team battles.

In [ ]:
not_teams = list(df_fp9.loc[df_fp9['is_teams'] == False].index)
not_teams = list(map(lambda filename: '../../data/Fight-Pitt-9/Game_' + filename + '.slp', not_teams))

In [ ]:
def get_not_teams():
    not_teams = list(df_fp9.loc[df_fp9['is_teams'] == False].index)
    return list(map(lambda filename: '../../data/Fight-Pitt-9/Game_' + filename + '.slp', not_teams))

I know I want to analyze matches that have a Fox, just because that is a character I personally play and know there are a lot of matches.

In [ ]:
df_fox = df_fp9.loc[(df_fp9['p1_char_name'] == 'Fox') | (df_fp9['p2_char_name'] == 'Fox')]
print('df_fp9.shape:', df_fp9.shape)
df_fox.shape

In [ ]:
sum(df_fp9['duration'])

In [ ]:
sum(df_fox['duration'])

I also want to drop all games that were not a 1 vs. 1 match. So, I want `is_teams` to be False.

In [ ]:
df_fox.shape

In [ ]:
df_fox = df_fox.loc[df_fox['is_teams'] == False]
df_fox

In [ ]:
sum(df_fox['duration'])

In [ ]:
df_fox['stage'].value_counts()

In [ ]:
stages = {
    2: 'Fountain of Dreams',
    3: 'Pokemon Stadium',
    4: "Princess Peach's Castle",
    5: 'Kongo Jungle',
    6: 'Brinstar',
    7: 'Corneria',
    8: "Yoshi's Story",
    9: 'Onett',
    10: 'Mute City',
    11: 'Rainbow Cruise',
    12: 'Jungle Japes',
    13: 'Great Bay',
    14: 'Hyrule Temple',
    15: 'Brinstar Depths',
    16: "Yoshi's Island",
    17: 'Green Greens',
    18: 'Fourside',
    19: 'Mushroom Kingdom I',
    20: 'Mushroom Kingdom II',
    22: 'Venom',
    23: 'Poke Floats',
    24: 'Big Blue',
    25: 'Icicle Mountain',
    26: 'Icetop',
    27: 'Flat Zone',
    28: 'Dream Land 64',
    29: "Yoshi's Island 64",
    30: 'Kongo Jungle 64',
    31: 'Battlefield',
    32: 'Final Destination'
}

In [ ]:
df_fox['stage_name'] = df_fox['stage'].map(stages)
df_fox

In [ ]:
df_fox['stage_name'].value_counts()

In [ ]:
# create a list of tuples where the first item is the stage name
# and the second item is the total number of frames played on that stage
# with at least one Fox
stage_frame_total = [(stage, sum(df_fox.loc[(df_fox['stage_name'] == stage), 'duration'])) for stage in set(df_fox['stage_name'].values)]
stage_frame_total

In [ ]:
sorted(stage_frame_total, key = lambda val: val[1])

Although there are more games played on Dream Land than any other stage, I will only be using games played on Final Destination. Between Dream Land, Final Destination has a frame difference of about 60,000 frames or 10,000 seconds or 

Now I want to get a frequency of each matchup. For example, how many matches did a Fox player fight another Fox player? How many matches did a Fox player fight a Captain Falcon?

In [ ]:
p1_nofox_counts = df_fox.loc[df_fox['p1_char_name'] != 'Fox', 'p1_char_name'].value_counts()
p2_nofox_counts = df_fox.loc[df_fox['p2_char_name'] != 'Fox', 'p2_char_name'].value_counts()
p1_nofox_counts

In [ ]:
p2_nofox_counts

In [ ]:
not_foxes = p2_nofox_counts.append(p1_nofox_counts)
not_foxes

In [ ]:
not_foxes.groupby(not_foxes.index).sum().sort_values(ascending = False)

Marth and Falco both have a lot of games against Fox. This makes sense because they both have strong options to defeat Fox. Let's drop all rows that do not have Fox, Falco, or Marth. Then consider how many frames exist a game is played on Final Destination with Fox vs. Marth or Fox vs. Falco.

In [ ]:
mask_fd = (df_fox['stage_name'] == 'Final Destination')
mask_marth = (df_fox['p1_char_name'] == 'Marth') | (df_fox['p2_char_name'] == 'Marth')
mask_falco = (df_fox['p1_char_name'] == 'Falco') | (df_fox['p2_char_name'] == 'Falco')

In [ ]:
df_falco_fd = df_fox.loc[mask_falco & mask_fd]
print(sum(df_falco_fd['duration']))
df_falco_fd.head()

In [ ]:
df_marth_fd = df_fox.loc[mask_marth & mask_fd]
print(sum(df_marth_fd['duration']))
df_marth_fd.head()

I will keep both dataframes, but will initially use `df_falco_fd` due to less frames since time is a current constraint.

Since I want to predict all of Fox's actions in this tournament when fighting against Falco, I will need to have a column that will specify which port is controlling Fox and which port is controlling Falco.

In [ ]:
df_falco_fd.loc[df_falco_fd['p1_char_name'] == 'Fox', 'p1_port'].shape

In [ ]:
df_falco_fd.loc[df_falco_fd['p2_char_name'] == 'Fox', 'p1_port'].shape

In [ ]:
df_falco_fd

In [ ]:
fox_ports = [df_falco_fd.loc[ident, 'p1_port'] if df_falco_fd.loc[ident, 'p1_char_name'] == 'Fox' else df_falco_fd.loc[ident, 'p2_port'] for ident in df_falco_fd.index]
fox_ports

In [ ]:
nfox_ports = [df_falco_fd.loc[ident, 'p1_port'] if df_falco_fd.loc[ident, 'p1_char_name'] != 'Fox' else df_falco_fd.loc[ident, 'p2_port'] for ident in df_falco_fd.index]
nfox_ports

In [ ]:
df_falco_fd['fox_ports'] = fox_ports
df_falco_fd['falco_ports'] = nfox_ports

In [ ]:
df_falco_fd

In [ ]:
falco_fd_gameid = list(df_falco_fd.index)
falco_fd_gameid

In [ ]:
falco_fd_games = list(map(lambda val: '../../data/Fight-Pitt-9/Game_' + val + '.slp', falco_fd_gameid))
falco_fd_games

In [ ]:
def frames_to_df(slp_paths):
    length = len(slp_paths)
    count = 0
    
    p1_button_dict = {'Trigger Analog':[],'Start': [],'Y': [],'X': [],'B': [],'A': [],'L': [],'R': [],
                      'Z': [],'Dpad-Up': [],'Dpad-Down': [],'Dpad-Right': [],'Dpad-Left': []}
    
    p2_button_dict = {'Trigger Analog':[],'Start': [],'Y': [],'X': [],'B': [],'A': [],'L': [],'R': [],
                      'Z': [],'Dpad-Up': [],'Dpad-Down': [],'Dpad-Right': [],'Dpad-Left': []}
    
    # foreign key to metadata dataframe
    game_id = list()
    
    # frame index
    index = list()
    
    # feature per frame for p1
    p1_combo_count, p1_dmg, p1_direction, p1_last_attack_landed, p1_last_hit_by, p1_position_x, \
    p1_position_y, p1_shield, p1_state, p1_stage_age, p1_stocks, p1_cstick_x, p1_cstick_y, \
    p1_joystick_x, p1_joystick_y, p1_state, p1_state_age = list(), list(), list(), list(), list(), \
    list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list()
    
    # feature per frame for p2
    p2_combo_count, p2_dmg, p2_direction, p2_last_attack_landed, p2_last_hit_by, p2_position_x, \
    p2_position_y, p2_shield, p2_state, p2_stage_age, p2_stocks, p2_cstick_x, p2_cstick_y, \
    p2_joystick_x, p2_joystick_y, p2_state, p2_state_age = list(), list(), list(), list(), list(), \
    list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list()
    
    for path in slp_paths:
        
        curr_gameid = slp_paths[count].split('/')[-1].strip('Game_').strip('.slp')
        count += 1
        print(f'Parsing file {count} of {length}: {round(count / length * 100, 2)}%')
        try:
            game = slp.Game(path)
        except:
            print(f'Skip game {count} of {length}')
            continue

        # get active ports
        # index 0: player 1
        # index 1: player 2
        ports = [game.start.players.index(port) for port in game.start.players if port != None]
        
        # for each Frame object of all frames in a specific game
        flength = len(game.frames)
        fcount = 0
        for frame in game.frames:
            fcount += 1
            print(f'Parsing frame {fcount} of {flength}: {round(fcount / flength * 100, 2)}%', end = '\r')
            game_id.append(curr_gameid)
            
            index.append(frame.index)
            try:
                p1_cstick_x.append(frame.ports[ports[0]].leader.pre.cstick.x)
            except:
                p1_cstick_x.append(None)
            
            try:
                p2_cstick_x.append(frame.ports[ports[1]].leader.pre.cstick.x)
            except:
                p2_cstick_x.append(None)

            try:
                p1_cstick_y.append(frame.ports[ports[0]].leader.pre.cstick.y)
            except:
                p1_cstick_y.append(None)

            try:
                p2_cstick_y.append(frame.ports[ports[1]].leader.pre.cstick.y)
            except:
                p2_cstick_y.append(None)

            try:
                p1_joystick_x.append(frame.ports[ports[0]].leader.pre.joystick.x)
            except:
                p1_joystick_x.append(None)
                
            try:
                p2_joystick_x.append(frame.ports[ports[1]].leader.pre.joystick.x)
            except:
                p2_joystick_x.append(None)
            
            try:
                p1_joystick_y.append(frame.ports[ports[0]].leader.pre.joystick.y)
            except:
                p1_joystick_y.append(None)

            try:
                p2_joystick_y.append(frame.ports[ports[1]].leader.pre.joystick.y)
            except:
                p2_joystick_y.append(None)

            try:
                p1_combo_count.append(frame.ports[ports[0]].leader.post.combo_count)
            except:
                p1_combo_count.append(None)
            try:
                p2_combo_count.append(frame.ports[ports[1]].leader.post.combo_count)
            except:
                p2_combo_count.append(None)

            try:
                p1_dmg.append(frame.ports[ports[0]].leader.post.damage)
            except:
                p1_dmg.append(None)
            try:
                p2_dmg.append(frame.ports[ports[1]].leader.post.damage)
            except:
                p2_dmg.append(None)

            try:
                p1_direction.append(frame.ports[ports[0]].leader.post.direction)
            except:
                p1_direction.append(None)
                
            try:
                p2_direction.append(frame.ports[ports[1]].leader.post.direction)
            except:
                p2_direction.append(None)

            try:
                p1_last_attack_landed.append(frame.ports[ports[0]].leader.post.last_attack_landed)
            except:
                p1_last_attack_landed.append(None)

            try:
                p2_last_attack_landed.append(frame.ports[ports[1]].leader.post.last_attack_landed)
            except:
                p2_last_attack_landed.append(None)
            
            try:
                p1_last_hit_by.append(frame.ports[ports[0]].leader.post.last_hit_by)
            except:
                p1_last_hit_by.append(None)
                
            try:
                p2_last_hit_by.append(frame.ports[ports[1]].leader.post.last_hit_by)
            except:
                p2_last_hit_by.append(None)
            
            try:
                p1_position_x.append(frame.ports[ports[0]].leader.post.position.x)
            except:
                p1_position_x.append(None)

            try:
                p2_position_x.append(frame.ports[ports[1]].leader.post.position.x)
            except:
                p2_position_x.append(None)
            
            try:
                p1_position_y.append(frame.ports[ports[0]].leader.post.position.y)
            except:
                p1_position_y.append(None)

            try:
                p2_position_y.append(frame.ports[ports[1]].leader.post.position.y)
            except:
                p2_position_y.append(None)

            try:
                p1_shield.append(frame.ports[ports[0]].leader.post.shield)
            except:
                p2_shield.append(None)

            try:
                p2_shield.append(frame.ports[ports[1]].leader.post.shield)
            except:
                p2_shield.append(None)

            try:
                p1_state.append(frame.ports[ports[0]].leader.post.state)
            except:
                p1_state.append(None)

            try:
                p2_state.append(frame.ports[ports[1]].leader.post.state)
            except:
                p2_state.append(None)

            try:
                p1_state_age.append(frame.ports[ports[0]].leader.post.state_age)
            except:
                p1_state_age.append(None)
            
            try:
                p2_state_age.append(frame.ports[ports[1]].leader.post.state_age)
            except:
                p2_state_age.append(None)

            try:
                p1_stocks.append(frame.ports[ports[0]].leader.post.stocks)
            except:
                p1_stocks.append(None)
                
            try:
                p2_stocks.append(frame.ports[ports[1]].leader.post.stocks)
            except:
                p2_stocks.append(None)
            try:
                p1_ins = str(frame.ports[ports[0]].leader.pre.buttons.logical).split('.')[1].split('|')
                for button in p1_button_dict:
                    if button in p1_ins:
                        p1_button_dict[button].append(1)
                    else:
                        p1_button_dict[button].append(0)
            except:
                for button in p1_button_dict:
                    p1_button_dict[button].append(None)
            try:
                p2_ins = str(frame.ports[ports[1]].leader.pre.buttons.logical).split('.')[1].split('|')
                for button in p2_button_dict:
                    if button in p2_ins:
                        p2_button_dict[button].append(1)
                    else:
                        p2_button_dict[button].append(0)
            except:
                for button in p2_button_dict:
                    p2_button_dict[button].append(None)

    return pd.DataFrame.from_dict({
        'game_id': game_id,
        'frame_index': index,
        
        # p1
        'p1_cstick_x': p1_cstick_x,
        'p1_cstick_y': p1_cstick_y,
        'p1_joystick_x': p1_joystick_x,
        'p1_joystick_y': p1_joystick_y,
        'p1_trigger_analog': p1_button_dict['Trigger Analog'],
        'p1_Start': p1_button_dict['Start'],
        'p1_Y': p1_button_dict['Y'],
        'p1_X': p1_button_dict['X'],
        'p1_B': p1_button_dict['B'],
        'p1_A': p1_button_dict['A'],
        'p1_L': p1_button_dict['L'],
        'p1_R': p1_button_dict['R'],
        'p1_Z': p1_button_dict['Z'],
        'p1_Dpad_Up': p1_button_dict['Dpad-Up'],
        'p1_Dpad_Down': p1_button_dict['Dpad-Down'],
        'p1_Dpad_Right': p1_button_dict['Dpad-Right'],
        'p1_Dpad_Left': p1_button_dict['Dpad-Left'],
        'p1_combo_count': p1_combo_count,
        'p1_dmg': p1_dmg,
        'p1_direction': p1_direction,
        'p1_last_attack_landed': p1_last_attack_landed,
        'p1_last_hit_by': p1_last_hit_by,
        'p1_position_x': p1_position_x,
        'p1_position_y': p1_position_y,
        'p1_shield': p1_shield,
        'p1_state': p1_state,
        'p1_state_age': p1_state_age,
        'p1_stocks': p1_stocks,
        
        # p2
        'p2_cstick_x': p2_cstick_x,
        'p2_cstick_y': p2_cstick_y,
        'p2_joystick_x': p2_joystick_x,
        'p2_joystick_y': p2_joystick_y,
        'p2_trigger_analog': p2_button_dict['Trigger Analog'],
        'p2_Start': p2_button_dict['Start'],
        'p2_Y': p2_button_dict['Y'],
        'p2_X': p2_button_dict['X'],
        'p2_B': p2_button_dict['B'],
        'p2_A': p2_button_dict['A'],
        'p2_L': p2_button_dict['L'],
        'p2_R': p2_button_dict['R'],
        'p2_Z': p2_button_dict['Z'],
        'p2_Dpad_Up': p2_button_dict['Dpad-Up'],
        'p2_Dpad_Down': p2_button_dict['Dpad-Down'],
        'p2_Dpad_Right': p2_button_dict['Dpad-Right'],
        'p2_Dpad_Left': p2_button_dict['Dpad-Left'],
        'p2_combo_count': p2_combo_count,
        'p2_dmg': p2_dmg,
        'p2_direction': p2_direction,
        'p2_last_attack_landed': p2_last_attack_landed,
        'p2_last_hit_by': p2_last_hit_by,
        'p2_position_x': p2_position_x,
        'p2_position_y': p2_position_y,
        'p2_shield': p2_shield,
        'p2_state': p2_state,
        'p2_state_age': p2_state_age,
        'p2_stocks': p2_stocks     
    }, orient = 'columns')

In [ ]:
len(get_not_teams())

In [ ]:
not_teams = get_not_teams()

There exists two games that are not team battles, but have more than two players. These games will not be included.

In [ ]:
slp.Game(not_teams[766]).start

In [ ]:
slp.Game(not_teams[292]).start.players

## USE `NOT_TEAMS.REMOVE(INDEX)`

In [ ]:
df_fp9_frames_01 = frames_to_df(not_teams[:101])
df_fp9_frames_02 = frames_to_df(not_teams[101:201])
df_fp9_frames_03 = frames_to_df(not_teams[201:292])
df_fp9_frames_03_5 = frames_to_df(not_teams[293:301])
df_fp9_frames_04 = frames_to_df(not_teams[301:401])
df_fp9_frames_05 = frames_to_df(not_teams[401:501])
df_fp9_frames_06 = frames_to_df(not_teams[501:601])
df_fp9_frames_07 = frames_to_df(not_teams[601:701])
df_fp9_frames_08 = frames_to_df(not_teams[701:766])
df_fp9_frames_08_5 = frames_to_df(not_teams[767:801])
df_fp9_frames_09 = frames_to_df(not_teams[801:901])
df_fp9_frames_10 = frames_to_df(not_teams[901:1001])
df_fp9_frames_11 = frames_to_df(not_teams[1001:1088])

Now to put all the above dataframes together as one.

In [ ]:
df_fp9_frames = pd.concat([df_fp9_frames_01, df_fp9_frames_02, df_fp9_frames_03, df_fp9_frames_03_5, df_fp9_frames_04,
          df_fp9_frames_05, df_fp9_frames_06, df_fp9_frames_07, df_fp9_frames_08, df_fp9_frames_08_5,
          df_fp9_frames_09, df_fp9_frames_10, df_fp9_frames_11])
print(df_fp9_frames.shape)
df_fp9_frames.isnull().sum()

In [ ]:
1512824 / 10589313

In [ ]:
df_fp9_frames.isnull().mean()

In [ ]:
def frames_to_df_fox(slp_paths):
    length = len(slp_paths)
    count = 0
    
    fox_button_dict = {'Trigger Analog':[],'Start': [],'Y': [],'X': [],'B': [],'A': [],'L': [],'R': [],
                      'Z': [],'Dpad-Up': [],'Dpad-Down': [],'Dpad-Right': [],'Dpad-Left': []}
    
    nfox_button_dict = {'Trigger Analog':[],'Start': [],'Y': [],'X': [],'B': [],'A': [],'L': [],'R': [],
                      'Z': [],'Dpad-Up': [],'Dpad-Down': [],'Dpad-Right': [],'Dpad-Left': []}
    
    # foreign key to metadata dataframe
    game_id = list()
    
    # frame index
    index = list()
    
    # feature per frame for fox
    fox_combo_count, fox_dmg, fox_direction, \
    fox_last_attack_landed, fox_last_hit_by, fox_position_x, fox_position_y, \
    fox_shield, fox_state, fox_stage_age, fox_stocks, fox_cstick_x, fox_cstick_y, fox_dmg, fox_direction, \
    fox_joystick_x, fox_joystick_y,  fox_position, fox_raw_analog_x, fox_state, fox_state_age = list(), list(), list(), \
    list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), \
    list(), list(), list(), list(), list(), list()
    
    # feature per frame for not fox
    nfox_combo_count, nfox_dmg, nfox_direction, \
    nfox_last_attack_landed, nfox_last_hit_by, nfox_position_x, nfox_position_y, \
    nfox_shield, nfox_state, p2_stage_age, nfox_stocks, nfox_cstick_x, nfox_cstick_y, nfox_dmg, nfox_direction, \
    nfox_joystick_x, nfox_joystick_y, nfox_position, nfox_raw_analog_x, nfox_state, nfox_state_age = list(), list(), list(), \
    list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), list(), \
    list(), list(), list(), list(), list(), list()
    
    for path in slp_paths:
        
        curr_gameid = slp_paths[count].split('/')[-1].strip('Game_').strip('.slp')
        print(f'Parsing file {count + 1} of {length}')
        try:
            game = slp.Game(path)
        except:
            print(f'Skip game {count + 1} of {length}')
            continue

        # get fox ports and non-fox ports
        fox_ports = [df_falco_fd.loc[ident, 'p1_port'] if df_falco_fd.loc[ident, 'p1_char_name'] == 'Fox' else df_falco_fd.loc[ident, 'p2_port'] for ident in df_falco_fd.index]
        nfox_ports = [df_falco_fd.loc[ident, 'p1_port'] if df_falco_fd.loc[ident, 'p1_char_name'] != 'Fox' else df_falco_fd.loc[ident, 'p2_port'] for ident in df_falco_fd.index]
        
        # for each Frame object of all frames in a specific game
        frame_length = len(game.frames)
        frame_count = 0
        for frame in game.frames:
            frame_count += 1
            print(f'Parsing frame {frame_count} of {frame_length}: {round(frame_count / frame_length * 100, 2)}%', end = '\r')
            game_id.append(curr_gameid)
            
            index.append(frame.index)
            
            fox_cstick_x.append(frame.ports[fox_ports[count]].leader.pre.cstick.x)
            nfox_cstick_x.append(frame.ports[nfox_ports[count]].leader.pre.cstick.x)

            fox_cstick_y.append(frame.ports[fox_ports[count]].leader.pre.cstick.y)
            nfox_cstick_y.append(frame.ports[nfox_ports[count]].leader.pre.cstick.y)
            
            fox_joystick_x.append(frame.ports[fox_ports[count]].leader.pre.joystick.x)
            nfox_joystick_x.append(frame.ports[nfox_ports[count]].leader.pre.joystick.x)
            
            fox_joystick_y.append(frame.ports[fox_ports[count]].leader.pre.joystick.y)
            nfox_joystick_y.append(frame.ports[nfox_ports[count]].leader.pre.joystick.y)
            
            fox_combo_count.append(frame.ports[fox_ports[count]].leader.post.combo_count)
            nfox_combo_count.append(frame.ports[nfox_ports[count]].leader.post.combo_count)
            
            fox_dmg.append(frame.ports[fox_ports[count]].leader.post.damage)
            nfox_dmg.append(frame.ports[nfox_ports[count]].leader.post.damage)
            
            fox_direction.append(frame.ports[fox_ports[count]].leader.post.direction)
            nfox_direction.append(frame.ports[nfox_ports[count]].leader.post.direction)
            
            fox_last_hit_by.append(frame.ports[fox_ports[count]].leader.post.last_hit_by)
            nfox_last_hit_by.append(frame.ports[nfox_ports[count]].leader.post.last_hit_by)
            
            fox_position_x.append(frame.ports[fox_ports[count]].leader.post.position.x)
            nfox_position_x.append(frame.ports[nfox_ports[count]].leader.post.position.x)
            
            fox_position_y.append(frame.ports[fox_ports[count]].leader.post.position.y)
            nfox_position_y.append(frame.ports[nfox_ports[count]].leader.post.position.y)
            
            fox_shield.append(frame.ports[fox_ports[count]].leader.post.shield)
            nfox_shield.append(frame.ports[nfox_ports[count]].leader.post.shield)
            
            fox_state.append(frame.ports[fox_ports[count]].leader.post.state)
            nfox_state.append(frame.ports[nfox_ports[count]].leader.post.state)
            
            fox_state_age.append(frame.ports[fox_ports[count]].leader.post.state_age)
            nfox_state_age.append(frame.ports[nfox_ports[count]].leader.post.state_age)
            
            fox_stocks.append(frame.ports[fox_ports[count]].leader.post.stocks)
            nfox_stocks.append(frame.ports[nfox_ports[count]].leader.post.stocks)
            
            fox_ins = str(frame.ports[fox_ports[count]].leader.pre.buttons.logical).split('.')[1].split('|')
            for button in fox_button_dict:
                if button in fox_ins:
                    fox_button_dict[button].append(1)
                else:
                    fox_button_dict[button].append(0)
            
            nfox_ins = str(frame.ports[nfox_ports[count]].leader.pre.buttons.logical).split('.')[1].split('|')
            for button in nfox_button_dict:
                if button in nfox_ins:
                    nfox_button_dict[button].append(1)
                else:
                    nfox_button_dict[button].append(0)
            
        count += 1
            
# p2_airborne, p2_combo_count, p2_dmg, p2_direction, p2_flags, p2_ground, p2_hit_stun, \
# p2_jumps, p2_Lcancel, p2_last_attack_landed, p2_last_hit_by, p2_position, p2_shield, \
# p2_state, p2_stage_age, p2_stocks, p2_cstick, p2_dmg, p2_direction, p2_joystick, \
# p2_position, p2_state

    return pd.DataFrame({
        'game_id': game_id,
        'frame_index': index,
        
        # p1
        'fox_cstick_x': fox_cstick_x,
        'fox_cstick_y': fox_cstick_y,
        'fox_joystick_x': fox_joystick_x,
        'fox_joystick_y': fox_joystick_y,
        'fox_trigger_analog': fox_button_dict['Trigger Analog'],
        'fox_Start': fox_button_dict['Start'],
        'fox_Y': fox_button_dict['Y'],
        'fox_X': fox_button_dict['X'],
        'fox_B': fox_button_dict['B'],
        'fox_A': fox_button_dict['A'],
        'fox_L': fox_button_dict['L'],
        'fox_R': fox_button_dict['R'],
        'fox_Z': fox_button_dict['Z'],
        'fox_Dpad_Up': fox_button_dict['Dpad-Up'],
        'fox_Dpad_Down': fox_button_dict['Dpad-Down'],
        'fox_Dpad_Right': fox_button_dict['Dpad-Right'],
        'fox_Dpad_Left': fox_button_dict['Dpad-Left'],
        'fox_combo_count': fox_combo_count,
        'fox_dmg': fox_dmg,
        'fox_direction': fox_direction,
        'fox_last_hit_by': fox_last_hit_by,
        'fox_position_x': fox_position_x,
        'fox_position_y': fox_position_y,
        'fox_shield': fox_shield,
        'fox_state': fox_state,
        'fox_state_age': fox_state_age,
        'fox_stocks': fox_stocks,
        
        # p2
        'nfox_cstick_x': nfox_cstick_x,
        'nfox_cstick_y': nfox_cstick_y,
        'nfox_joystick_x': nfox_joystick_x,
        'nfox_joystick_y': nfox_joystick_y,
        'nfox_trigger_analog': nfox_button_dict['Trigger Analog'],
        'nfox_Start': nfox_button_dict['Start'],
        'nfox_Y': nfox_button_dict['Y'],
        'nfox_X': nfox_button_dict['X'],
        'nfox_B': nfox_button_dict['B'],
        'nfox_A': nfox_button_dict['A'],
        'nfox_L': nfox_button_dict['L'],
        'nfox_R': nfox_button_dict['R'],
        'nfox_Z': nfox_button_dict['Z'],
        'nfox_Dpad_Up': nfox_button_dict['Dpad-Up'],
        'nfox_Dpad_Down': nfox_button_dict['Dpad-Down'],
        'nfox_Dpad_Right': nfox_button_dict['Dpad-Right'],
        'nfox_Dpad_Left': nfox_button_dict['Dpad-Left'],
        'nfox_combo_count': nfox_combo_count,
        'nfox_dmg': nfox_dmg,
        'nfox_direction': nfox_direction,
        'nfox_last_hit_by': nfox_last_hit_by,
        'nfox_position_x': nfox_position_x,
        'nfox_position_y': nfox_position_y,
        'nfox_shield': nfox_shield,
        'nfox_state': nfox_state,
        'nfox_state_age': nfox_state_age,
        'nfox_stocks': nfox_stocks
    })

In [ ]:
test = frames_to_df_fox(falco_fd_games[:3])
test.head()

In [ ]:
test.shape

In [ ]:
test.isnull().sum()

[This shows all the unique state values of each character](https://docs.google.com/spreadsheets/d/1Nu3hSc1U6apOhU4JIJaWRC4Lj0S1inN8BFsq3Y8cFjI/preview#gid=1072939422)

Values 341 to 375 need to be altered so that they are represented as their own unique values. Since there are 382 action states that are applicable to all characters, then we must get Fox's action state value of 341 to change to 383 and Fox's action state value of 375 to 417. In short, values within the range of $[341, 375]$ must have 42 added to themselves.

In [ ]:
test['fox_state'] = test['fox_state'].map(lambda val: val if (val < 341) or (val > 375) else val + 42)

In [ ]:
set(test['fox_state'].values)

The largest action state value is 411, this is okay because the values that we expect to be $[412, 417]$ are the action states of Fox's taunt. These are generally not used because taunting in competitive play is poor sportsmanship. Although there are niche scenarios in which taunting is beneficial in combat, it is not a frequent tactic in the current metagame as far as I know.(Refering to KJH's taunt-downsmash edgegaurd on Marth's).

Now the same idea must be applied for Falco. Now that Fox's unique states bring the number of total action states to 417, then Falco's unique state values within the range of $[341, 375]$ should become $[418, 452]$. This is done by adding 77 to each of Falco's unique animation states.

It should be noted that although Fox and Falco have nearly identical animations, the properties of each attack's hitbox's are different.

In [ ]:
test['nfox_state'] = test['nfox_state'].map(lambda val: val if (val < 341) or (val > 375) else val + 77)

In [ ]:
test

In [ ]:
foo = [0,1,2,3,4]
foo[:5]

In [ ]:
df_fp9_frames.drop(columns = ['p1_Lcancel', 'p2_Lcancel'], inplace = True)

In [ ]:
df_fp9_frames.shape

In [ ]:
df_fp9_frames.shape

In [ ]:
df_ff_frames = frames_to_df_fox(falco_fd_games)

In [ ]:
df_ff_frames.to_csv('../data/fp9_frames.csv')

In [ ]:
df_ff_frames.shape[0] == sum(df_falco_fd['duration'])

In [ ]:
df_ff_frames['fox_state'] = df_ff_frames['fox_state'].map(lambda val: val if (val < 341) or (val > 375) else val + 42)

In [ ]:
df_ff_frames['nfox_state'] = df_ff_frames['nfox_state'].map(lambda val: val if (val < 341) or (val > 375) else val + 77)

In [ ]:
df_ff_frames.reset_index(inplace = True, drop = True)

In [ ]:
df_ff_frames.columns

In [ ]:
set(df_ff_frames['nfox_trigger_analog'].values)

In [ ]:
set(df_ff_frames['fox_trigger_analog'].values)

In [ ]:
set(df_ff_frames['fox_Start'].values)

In [ ]:
set(df_ff_frames['nfox_Start'].values)

In [ ]:
set(df_ff_frames['nfox_Dpad_Up'].values)

In [ ]:
set(df_ff_frames['nfox_Dpad_Down'].values)

In [ ]:
set(df_ff_frames['nfox_Dpad_Left'].values)

In [ ]:
set(df_ff_frames['nfox_Dpad_Right'].values)

In [ ]:
set(df_ff_frames['fox_Dpad_Up'].values)

In [ ]:
set(df_ff_frames['fox_Dpad_Down'].values)

In [ ]:
set(df_ff_frames['fox_Dpad_Left'].values)

In [ ]:
set(df_ff_frames['fox_Dpad_Right'].values)

In [ ]:
set(df_ff_frames['fox_stocks'].values)

In [ ]:
df_ff_frames.drop(columns = ['fox_trigger_analog', 'nfox_trigger_analog', 'fox_Start', 'nfox_Start',
                             'fox_Dpad_Up', 'fox_Dpad_Down', 'fox_Dpad_Left', 'fox_Dpad_Right', 'nfox_Dpad_Up',
                            'nfox_Dpad_Down', 'nfox_Dpad_Left', 'nfox_Dpad_Right'], inplace = True)

In [ ]:
df_ff_frames

Map all missed tech action states to a single value

In [ ]:
missed_tech = [183, 188, 189, 191, 196, 197]

In [ ]:
df_ff_frames['fox_state'] = df_ff_frames['fox_state'].apply(lambda val: val if val not in missed_tech else 999)
df_ff_frames.head()

In [ ]:
df_ff_frames.loc[df_ff_frames['fox_state'] == 999]

In [ ]:
df_ff_frames.loc[df_ff_frames['fox_state'] == 199]

In [ ]:
df_ff_frames.loc[df_ff_frames['fox_state'] == 200]

In [ ]:
df_ff_frames.loc[df_ff_frames['fox_state'] == 201]

In [ ]:
set(df_ff_frames['fox_state'])

In [ ]:
df_ff_frames.loc[df_ff_frames['fox_state'] == 199, ['game_id', 'frame_index', 'fox_state']].head()

## Extracting Frame Data from Games 

In [ ]:
fight_pitt_9[215]

In [ ]:
game = slp.Game(fight_pitt_9[215])
game.metadata.duration

In [ ]:
frames = game.frames
frames

In [ ]:
frames[0].ports[2].leader.post.state

In [ ]:
frames[60].ports[2].leader.pre.buttons

In [ ]:
frames[60].ports[2].leader.pre.triggers

In [ ]:
# def remove_init_frames(game):
#     return [frame for frame in frames if frame.index >= 0]

In [ ]:
# frames = remove_init_frames(game)

### Index

In [ ]:
frames[0].index

## Player 1 Information

### Direction

In [ ]:
frames[0].ports[3]

### 

In [ ]:
for i in range(len(frames)):
    if frames[i].ports[3].leader.post.state == 191:
        print(i)
        print(frames[i].ports[3].leader.post.state)
        print()

In [ ]:
frames[0].ports

In [ ]:
frames[0].ports[df_falco_fd.loc[213, 'p1_port']]

In [ ]:
ic_game = slp.Game(fight_pitt_9[0])

In [ ]:
ic_game.frames[0].ports[0]

In [ ]:
ic_game.start

In [ ]:
for i in ic_game.frames[200:300]:
    print(i.ports[3].leader.pre.buttons)

In [ ]:
ic_game.frames[300].ports[3].leader.pre.buttons

In [ ]:
df_fp9.loc[df_fp9['p1_char_name'] == 'Ice Climbers']

In [ ]:
df_falco_fd.loc[213, 'p1_port']

### Get active ports

This will be taken from the df_falco_fd or df_marth_fd dataframe.

In [ ]:
df_falco_fd.head()

In [ ]:
df_falco_fd.loc[213, 'p1_port']

In [ ]:
df_falco_fd.loc[213, 'p2_port']

In [ ]:
frames[0].ports[0]

In [ ]:
frames[0].ports[3].leader.post.position

In [ ]:
frames[0]

In [ ]:
game.metadata.duration

In [ ]:
test_game.start

In [ ]:
frame = test_game.frames[0]
frame

In [ ]:
frame.ports[0].leader.post

In [ ]:
frame.ports[0].leader.pre

In [ ]:
frame.ports[0].leader.post.

In [ ]:
pd.Series({})

In [ ]:
frame.index

In [ ]:
len(other_game.frames)

In [ ]:
count = 0
for i in range(len(other_game.frames)):
    if other_game.frames[i].ports[1].leader.post.last_attack_landed != None:
#         print(other_game.frames[i].ports[1].leader.post)
        count += 1
print(count)

In [ ]:
frame.ports

```python
def metadata_to_df(slp_paths):
    '''
    Of a collection of games, store the metadata as a dataframe.
    
    slp_paths (list): each value is the file path to games
    returns a dataframe
    '''
    dates, durations, plats, p1_ports, p1_chars, p2_ports, p2_chars = list(), list(), list(), list(), list(), list(), list()
    if len(slp_paths) == 1:
        game = slp.Game(slp_paths[0])
        ports = [game.metadata.players.index(port) for port in game.metadata.players if port != None]
        characters = [char for port in ports for char in game.metadata.players[port].characters]
        try:
            return pd.Series({
                'date': game.metadata.date,
                'duration': game.metadata.duration,
                'platform': game.metadata.platform,
                'p1_port': ports[0],
                'p1_char': characters[0],
                'p2_port': ports[1],
                'p2_char': characters[1]
            })
        except:
            return pd.Series({
                'date': game.metadata.date,
                'duration': game.metadata.duration,
                'platform': game.metadata.platform,
                'p1_port': '',
                'p1_char': '',
                'p2_port': '',
                'p2_char': ''
            })
    else:
        for path in slp_paths:
            try:
                game = slp.Game(path)
            except:
                continue
            dates.append(game.metadata.date)
            durations.append(game.metadata.duration)
            plats.append(game.metadata.platform)

            # get active ports
            ports = [game.metadata.players.index(port) for port in game.metadata.players if port != None]
            try:
                p1_ports.append(ports[0])
                p2_ports.append(ports[1])
            except:
                p1_ports.append('')
                p2_ports.append('')

            # get characters
            characters = [char for port in ports for char in game.metadata.players[port].characters]
            try:
                p1_chars.append(characters[0])
                p2_chars.append(characters[1])
            except:
                p1_chars.append('')
                p2_chars.append('')
        return pd.DataFrame(data = {
                'date': dates,
                'duration': durations,
                'platform': plats,
                'p1_port': p1_ports,
                'p1_char': p1_chars,
                'p2_port': p2_ports,
                'p2_char': p2_chars
            })
    return None```

In [ ]:
slp.Game(fight_pitt_9[0]).start

In [ ]:
df_fp9.tail()

### Data Dictionary



I want each row to be a single frame. Columns will be
- binary controller inputs: A, B, X, Y, Z, Directional Pad, Start.
- Continuous(?) inputs from 0 to 1: L and R
- Continuous(?) inputs from -1 to 1 on each axis: Analog stick and C-stick
- stage (`game.start.stage`)
- position of selected player (`game.frames[n].ports[m].leader.pre.position`)
- direction the selected player is facing (

Do I want a dataframe per player per game?

Do I want to use pre-frame or post-frame information?



In [ ]:
slp.id.ActionState.ENTRY_START

In [ ]:
def after_entry(game):
    for frame in game.frames:
        if frame.ports[1].leader.pre.state != slp.id.ActionState.ENTRY_START \
        and frame.ports[1].leader.pre.state != slp.id.ActionState.ENTRY_END \
        and frame.ports[1].leader.pre.state != slp.id.ActionState.ENTRY:
            return frame
after_entry(game).ports[0].leader.pre.state

In [ ]:
game.frames[0].ports[1].leader.pre.state

In [ ]:
game.frames[20].ports[1].leader.pre.state

In [ ]:
game.frames[0].ports[1].leader.pre.position

In [ ]:
game.frames[2354].ports[1].leader.pre.buttons

In [ ]:
game.frames[0].ports[1].leader.pre.buttons

In [ ]:
game.frames[0].ports[1].leader.post

In [ ]:
game.frames[0]

In [ ]:
frame_index = [frame.index for frame in frames]

In [ ]:
frames[0].ports

In [ ]:
for frame in game.frames:
    data = frame.ports[0].leader # see also: port.follower (ICs)
    print(data.post.state) # character's post-frame action state

In [ ]:
game.frames[700].ports[0].leader.post.ground

In [ ]:
for i in range(len(game.frames)):
    if game.frames[i].ports[3].leader.post.ground != None:
        print(i)

<a id='terms'></a>
## Terminology
- Melee (noun): Shorthand for Super Smash Bros. Melee
- Stock (noun): A single unit of character lives. When a character runs out of stocks, then they lose the game.
- Frame (noun): A single unit of time for animation. In this case, there are 60 frames per second.
    - If a video is a collection of images being shown sequentially at a specified rate, then a single image is a frame.
- Hurtbox (noun): An invisible shape that contours the visible character model. Allows interaction between various elements of the game such as hitboxes.
- Hitbox (noun): An invisible shape that, when collided with an opponent's hurtbox, the game registers a hit.
- Mixup (noun): Changing one's pattern of fighting to be unpredictable and gain advantage over the opponent.
- Fox (noun): A playable character in Super Smash Bros. Melee.
- Marth (noun): 
- Stage (noun): The stage the characters are fighting on.
- Blast Zone (noun): Outer perimeter of the stage. When a character crosses over the perimeter, then that character loses a stock.
- Tech (verb): When a character is launched towards a surface, they can press either L or R triggers within 20 frames of colliding with the surface in order to be able to retaliate faster than if one were to not tech.
    - When teching on the ground, a character can tech in place, tech-roll left, tech-roll right, or not tech at all.
![tech-example](../images/tech.gif) <center>Example of a neutral tech or tech in-place</center>

In [ ]:
avg inputs per second fox n marth,
kd ratio fox vs marth, 
w/l ratio fox vs marth,
avg damage per stock fox vs marth,


WHEN MODELING
try sklearn.model_selection.RandomizedSearchCV